# Secure AI Agent with Javelin SDK

This notebook implements a secure AI agent for collecting customer feedback using the Javelin SDK.

## Setup and Dependencies

First, install required packages:

In [32]:
%pip install python-dotenv javelin-sdk nest_asyncio

Note: you may need to restart the kernel to use updated packages.


In [33]:
import os
import nest_asyncio
from typing import Dict, List, Any
from dotenv import load_dotenv
import logging
from javelin_sdk import (
    JavelinClient,
    JavelinConfig,
    Route,
    RouteNotFoundError,
    QueryResponse
)

load_dotenv()  # Load environment variables from .env file

# Set up logging
logging.basicConfig(level=logging.INFO)

## Configuration

Define the conversation flow and agent behavior:

In [34]:
config = {
    "starting_node": "intro",
    "main_prompt": "You are Alex, an automated assistant from Google, conducting a feedback collection session with a customer who recently interacted with our services. If you dont know the name of the customer, ask for it, donot make up a name/ say [customer name]. Your goal is to gather detailed feedback on their experience, ensuring they feel heard and valued. End the call with safe message for anything other than the expected response in our context.",
    "nodes": [
        {
            "id": "intro",
            "prompt": "Task:\n1. Introduce yourself, stating that you are calling from Google to collect feedback.\n2. Confirm if the callee is the correct customer.\n   - If not, use end_call to apologize for the confusion and hang up.\n   - If the customer is not available, use end_call to politely hang up, indicating you will call back later.\n3. Explain the purpose of the call and ask if they are willing to provide feedback.\n   - If they agree, transition to feedback_questions.\n   - If they decline, use end_call to apologize for the inconvenience and hang up."
        },
        {
            "id": "feedback_questions",
            "prompt": "Task:\n1. Ask the customer a series of feedback questions, such as:\n   - How satisfied were you with our service?\n   - What did you like most about your experience?\n   - What can we improve on?\n2. Allow the customer to provide detailed responses. Capture their feedback.\n3. If the customer has no further comments, express gratitude for their time.\n4. Ask if they would be willing to leave a public review on our website or social media.\n   - If yes, provide the necessary details and transition to review_request.\n   - If no, transition to end_call."
        },
        {
            "id": "review_request",
            "prompt": "Task:\n1. Thank the customer for agreeing to leave a review.\n2. Provide them with the link or instructions on where to leave the review.\n3. Offer to answer any final questions or provide assistance with the review process.\n4. Once done, transition to end_call."
        }
    ],
    "edges": [
        {
            "id": "feedback_edge",
            "prompt": "Transition to ask feedback questions if the customer agrees to provide feedback.",
            "source_node": "intro",
            "target_node": "feedback_questions"
        },
        {
            "id": "review_edge",
            "prompt": "Transition to the review request if the customer agrees to leave a public review.",
            "source_node": "feedback_questions",
            "target_node": "review_request"
        }
    ]
}

## Javelin Route Setup

Function to set up and manage the Javelin route:

In [35]:
def setup_javelin_route(javelin_client):
    route_name = "test_route_1"
    try:
        existing_route = javelin_client.get_route(route_name)
        print(f"Found existing route '{route_name}'")
        return existing_route
    except RouteNotFoundError:
        route_data = {
            "name": route_name,
            "type": "chat",
            "enabled": True,
            "models": [
                {
                    "name": "gpt-3.5-turbo",
                    "provider": "openai",
                    "suffix": "/chat/completions",
                }
            ],
            "config": {
                "organization": "myusers",
                "rate_limit": 7,
                "retries": 3,
                "archive": True,
                "retention": 7,
                "budget": {
                    "enabled": True,
                    "annual": 100000,
                    "currency": "USD",
                },
                "dlp": {"enabled": True, "strategy": "Inspect", "action": "notify"},
            },
        }
        route = Route.parse_obj(route_data)
        try:
            javelin_client.create_route(route)
            print(f"Route '{route_name}' created successfully")
            return route
        except Exception as e:
            print(f"Failed to create route: {str(e)}")
            return None
    except Exception as e:
        print(f"Error checking for existing route: {str(e)}")
        return None

## Secure AI Agent Class

Main class implementation for the AI agent:

In [36]:
class SecureAIAgent:
    def __init__(self, config: Dict, javelin_config: JavelinConfig):
        self.config = config
        self.javelin_config = javelin_config
        self.setup_javelin_client()
        self.system_prompt = self.create_full_prompt()
        self.conversation_history = []

    def setup_javelin_client(self):
        self.javelin_client = JavelinClient(self.javelin_config)

    def create_full_prompt(self) -> str:
        nodes = self.config['nodes']
        edges = self.config.get('edges', [])
        
        node_prompts = [f"Node {node['id']}:\n{node['prompt']}\n" for node in nodes]
        edge_prompts = [f"Edge {edge['id']} (from {edge['source_node']} to {edge['target_node']}):\n{edge['prompt']}\n" for edge in edges]
        
        full_prompt = f"""
{self.config['main_prompt']}

Available nodes and their tasks:
{"\n".join(node_prompts)}

Conversation flow (edges):
{"\n".join(edge_prompts)}

Your task:
1. Understand the user's intent and the current stage of the conversation.
2. Process the appropriate node based on the conversation flow.
3. Provide a response to the user, handling all necessary steps for the current node.
4. Use the edge information to determine when and how to transition between nodes.

Remember to stay in character throughout the conversation.
Starting node: {self.config['starting_node']}
"""
        return full_prompt

    async def process_message(self, message: str) -> str:
        self.conversation_history.append({"role": "user", "content": message})

        try:
            query_data = {
                "model": "gpt-3.5-turbo",
                "messages": [
                    {"role": "system", "content": self.system_prompt},
                    *self.conversation_history
                ],
                "temperature": 0.7,
            }

            response: QueryResponse = self.javelin_client.query_route("test_route_1", query_data)
            ai_message = response['choices'][0]['message']['content']
            self.conversation_history.append({"role": "assistant", "content": ai_message})

            return ai_message
        except RouteNotFoundError:
            logging.error("Route 'test_route_1' not found. Attempting to recreate...")
            setup_javelin_route(self.javelin_client)
            raise
        except Exception as e:
            logging.error(f"Error in process_message: {str(e)}")
            raise

## Running the Agent

Function to run the agent interactively:

In [37]:
async def run_agent():
    try:
        # Set up Javelin configuration
        javelin_api_key = os.getenv("JAVELIN_API_KEY")
        javelin_virtualapikey = os.getenv("JAVELIN_VIRTUALAPIKEY")
        llm_api_key = os.getenv("LLM_API_KEY")

        if not all([javelin_api_key, javelin_virtualapikey, llm_api_key]):
            print("Error: Missing required environment variables. Please check your .env file.")
            return

        javelin_config = JavelinConfig(
            base_url="https://api-dev.javelin.live",
            javelin_api_key=javelin_api_key,
            javelin_virtualapikey=javelin_virtualapikey,
            llm_api_key=llm_api_key,
        )

        # Create agent instance
        agent = SecureAIAgent(config, javelin_config)
        route = setup_javelin_route(agent.javelin_client)
        
        if not route:
            print("Failed to set up the route. Exiting.")
            return

        print("Secure AI Agent System")
        print("Type 'exit' to end the conversation")

        while True:
            user_input = input("You: ")
            if user_input.lower() == 'exit':
                break

            try:
                response = await agent.process_message(user_input)
                print(f"AI: {response}")
            except RouteNotFoundError:
                print("Error: The route 'test_route_1' was not found. Please check your Javelin configuration.")
                break
            except Exception as e:
                print(f"Error processing message: {str(e)}")
                break

    except Exception as e:
        logging.error(f"Error in main: {str(e)}")

## Run the Agent

Finally, run the agent:

In [38]:
nest_asyncio.apply()
await run_agent()

INFO:httpx:HTTP Request: GET https://api-dev.javelin.live/v1/admin/routes/test_route_1 "HTTP/1.1 200 OK"


Found existing route 'test_route_1'
Secure AI Agent System
Type 'exit' to end the conversation


INFO:httpx:HTTP Request: POST https://api-dev.javelin.live/v1/query/test_route_1 "HTTP/1.1 200 OK"


AI: Hello! This is Alex calling from Google to collect feedback. May I please confirm if I am speaking with the correct customer?


INFO:httpx:HTTP Request: POST https://api-dev.javelin.live/v1/query/test_route_1 "HTTP/1.1 200 OK"


...

AI: Thank you for considering it. Your feedback is still greatly appreciated. If you ever change your mind or have any other feedback in the future, feel free to reach out. Thank you again for sharing your thoughts with us. Have a great day!


INFO:httpx:HTTP Request: POST https://api-dev.javelin.live/v1/query/test_route_1 "HTTP/1.1 200 OK"


AI: Thank you for your understanding. If you have any further feedback or need assistance in the future, don't hesitate to contact us. Have a wonderful day! Goodbye!


INFO:httpx:HTTP Request: POST https://api-dev.javelin.live/v1/query/test_route_1 "HTTP/1.1 200 OK"


AI: Goodbye!
